In [1]:
from MT1D import MT1DProblem, MT1DSurvey, MT1DSrc, ZxyRx, Survey, AppResPhaRx
from SimPEG import Maps, DataMisfit
from scipy.constants import mu_0
import numpy as np

In [2]:
rxloc = np.r_[0.]
srcloc = np.r_[0.]
frequency = np.logspace(-3, 2, 25)
rxType = "AppredPha"

if rxType == "Zxy":
    rx = ZxyRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
elif rxType == "AppredPha":
    rx = AppResPhaRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
    
src = MT1DSrc(rxList, loc=srcloc)
survey = MT1DSurvey([src])
mesh = survey.setMesh(sigma=0.01, max_depth_core=5000., ncell_per_skind=10, n_skind=2)
sigma = np.ones(mesh.nC) * 0.01

prob = MT1DProblem(mesh, sigmaMap=Maps.ExpMap(mesh), verbose=False)
prob.pair(survey)

>> Smallest cell size = 50 m
>> Padding distance = 316227 m
>> # of padding cells 28


In [3]:
%%time
f = prob.fields(np.log(sigma))
pred = survey.dpred(np.log(sigma), f=f)

CPU times: user 82.6 ms, sys: 3.32 ms, total: 85.9 ms
Wall time: 86.3 ms


In [4]:
%%time
prob.Jvec(np.log(sigma), np.log(sigma), f=f)

CPU times: user 70.2 ms, sys: 3.39 ms, total: 73.6 ms
Wall time: 77.2 ms


array([ 582.89818726,    0.        ,  475.79213142,    0.        ,
        452.17393127,    0.        ,  463.0933006 ,    0.        ,
        464.37256158,    0.        ,  463.95086458,    0.        ,
        463.81894686,    0.        ,  463.63306051,    0.        ,
        463.39304316,    0.        ,  463.08204436,    0.        ,
        462.67928177,    0.        ,  462.1618593 ,    0.        ,
        461.51177709,    0.        ,  460.73191407,    0.        ,
        459.8715224 ,    0.        ,  459.04617148,    0.        ,
        458.40733681,    0.        ,  458.01383075,    0.        ,
        457.68287866,    0.        ,  457.09477865,    0.        ,
        456.15117445,    0.        ,  454.94828761,    0.        ,
        453.44345561,    0.        ,  451.53016592,    0.        ,
        449.10211943,    0.        ])

In [5]:
%%time
prob.Jtvec(np.log(sigma), pred)

CPU times: user 177 ms, sys: 5.37 ms, total: 182 ms
Wall time: 192 ms


array([   23.79611596,   312.61768115,   286.10622208,  -327.47627237,
       -1389.92601879, -2640.14857617, -3855.61971672, -4898.99800561,
       -5706.68155523, -6262.48979016, -6575.56008121, -6666.78851932,
       -6562.32658863, -6291.38040797, -5885.882515  , -5380.29417842,
       -4810.55169101, -4211.95167808, -3616.44262841, -3050.12969669,
       -2531.73320977, -2072.31466116, -1676.13019245, -1342.170202  ,
       -1065.8961971 ,  -840.81053945,  -659.67328181,  -515.32935714,
        -401.19747075,  -405.45027849,  -409.78992909,  -414.21911674,
        -418.74064842,  -423.35744986,  -428.07257191,  -432.8891973 ,
        -437.81064785,  -442.84039215,  -447.98205378,  -453.23942005,
        -458.61645132,  -464.11729104,  -469.74627638,  -475.50794971,
        -481.40707087,  -487.44863027,  -493.63786308,  -499.98026438,
        -506.48160549,  -513.14795158,  -519.98568062,  -527.00150388,
        -534.20248799,  -541.59607879,  -549.19012722,  -556.99291716,
      

In [6]:
from SimPEG import Tests
def derChk(m):
    return [survey.dpred(m), lambda mx: prob.Jvec(m, mx)]
Tests.checkDerivative(derChk, np.log(sigma), plotIt=False, num=3, eps=1e-20, dx=np.log(sigma)*2)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    7.847e+02     3.216e+02      nan
 1   1.00e-02    4.885e+01     2.343e+00      2.137
 2   1.00e-03    4.676e+00     3.227e-02      1.861
========================= PASS! =========================
That was easy!



True

In [7]:
survey.dobs = pred
dmis = DataMisfit.l2_DataMisfit(survey)
m0 = np.log(sigma)*2.

SimPEG.DataMisfit.l2_DataMisfit assigning default eps of 1e-5 * ||dobs||


In [8]:
Tests.checkDerivative(
    lambda m: [dmis(m), dmis.deriv(m)],
    m0,
    plotIt=False,
    num=4,
    dx = m0*1.
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.126e+08     1.347e+08      nan
 1   1.00e-02    8.515e+06     7.286e+05      2.267
 2   1.00e-03    7.856e+05     6.910e+03      2.023
 3   1.00e-04    7.794e+04     6.907e+01      2.000
========================= PASS! =========================
Not just a pretty face Sgkang



True

In [9]:
v = np.random.rand(mesh.nC)
w = np.random.rand(pred.shape[0])
wtJv = w.dot(prob.Jvec(m0, v))
vtJtw = v.dot(prob.Jtvec(m0, w))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 1.4551915228366852e-11, True)
